### 1 Fitting 

- Fitting with experimental data of DART group
- All parameters are fixed between two states.
- Target points are six: three rEs and three rSs in DART group
- Parameters are five weights

    wEE_star, wES_star, wSE, wSE_star, wSS_star
- External inputs are caluculated with data of control group

    - rE(l,m,h) = gain_E*(wEE_star * rE(l,m,h) - wES_star * r(l,m,h) + IE(l,m,h))^2
    - rS(l,m,h) = gain_S*((wSE - wSE_star) * rE(l,m,h) + wSS_star * rS(l,m,h) + IS(l,m,h))^2

In [ ]:
'''
This script is for case that all parameters are fixed
'''

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

# 定义实验数据
rEdatacontrol = np.array([[0.1012, 0.1032, 0.1254], [0.1824, 0.1939, 0.2282]])
rEdataDART = np.array([[0.1190, 0.1304, 0.1593], [0.2021, 0.2235, 0.2665]])
rSdatacontrol = np.array([[0.0539, 0.0728, 0.1014], [0.1285, 0.1688, 0.2091]])
rSdataDART = np.array([[0.0246, 0.0463, 0.0883], [0.1116, 0.1717, 0.2423]])
rEseDART = np.array([[0.007428755604043545,0.007788206032446528,0.009314834901506188], 
                     [0.01152857600761969,0.011741811713686944,0.013709086399540904]])
rSseDART = np.array([[0.004272683280978827,0.006342322667567268,0.010546259821164696], 
                     [0.013669393588839406,0.015771868172978647,0.021288306730872463]])

alphaE = 0.25 / rEdatacontrol[1, 1]
alphaS = 0.25 / rSdatacontrol[1, 1]

contrast = [0.25, 0.5, 1.0]

best_results = []

def cost_function(params, x):
    WEE, WES, WSE, WSVE, WSS = params

    IE = np.sqrt(rEdatacontrol[1, 2] / alphaE) - WEE * rEdatacontrol[1,2] + WES * rSdatacontrol[1,2]
    IS = np.sqrt(rSdatacontrol[1,2] / alphaS) - (WSE - WSVE) * rEdatacontrol[1,2] - WSS * rSdatacontrol[1,2]
    rE = rEdatacontrol[1,2]
    rS = rSdatacontrol[1,2]
    rE_values = []
    rS_values = []

    for it in range(total_duration):
        if rE >= 10.0 or rS >= 10.0:
            return 100
        drE = -rE + alphaE * (WEE * rE - WES * rS + IE) ** 2
        drS = -rS + alphaS * (((1.0 - x) * WSE - WSVE) * rE + WSS * rS + IS) ** 2
        rE += 0.1 * drE
        rS += 0.1 * drS
        rE_values.append(rE)
        rS_values.append(rS)

    if len(rE_values) == total_duration and len(rS_values) == total_duration:
        rE_max_first_half = max(rE_values[:total_duration//2])
        rE_max_second_half = max(rE_values[total_duration//2:])
    else:
        return 100

    rE_avg = sum(rE_values[total_duration//2:]) / (total_duration//2)

    if np.abs(rE_avg - rEdatacontrol[1, 2]) > 1e-4:
        print('here')
        return 100

    cost = 0.0

    for istate in range(2):
        for icontrast in range(3):
            IE = np.sqrt(rEdatacontrol[istate, icontrast] / alphaE) - WEE * rEdatacontrol[istate, icontrast] + WES * rSdatacontrol[istate, icontrast]
            IS = np.sqrt(rSdatacontrol[istate, icontrast] / alphaS) - (WSE - WSVE) * rEdatacontrol[istate, icontrast] - WSS * rSdatacontrol[istate, icontrast]
            rE = rEdataDART[istate, icontrast]
            rS = rSdataDART[istate, icontrast]
            rE_values = []
            rS_values = []

            for it in range(total_duration):
                if rE >= 10.0 or rS >= 10.0:
                    return 100
                drE = -rE + alphaE * (WEE * rE - WES * rS + IE) ** 2
                drS = -rS + alphaS * (((1.0 - x) * WSE - WSVE) * rE + WSS * rS + IS) ** 2
                rE += 0.1 * drE
                rS += 0.1 * drS
                rE_values.append(rE)
                rS_values.append(rS)
        
            if len(rE_values) == total_duration and len(rS_values) == total_duration:
                rE_max_first_half = max(rE_values[:total_duration//2])
                rE_max_second_half = max(rE_values[total_duration//2:])
            else:
                return 100
            
            rE_avg = sum(rE_values[total_duration//2:]) / (total_duration//2)
            rS_avg = sum(rS_values[total_duration//2:]) / (total_duration//2)
        
            cost += ((rE_avg - rEdataDART[istate, icontrast])/rEseDART[istate, icontrast]) ** 2 + \
                ((rS_avg - rSdataDART[istate, icontrast])/rSseDART[istate, icontrast]) ** 2
    
    return cost


total_duration = 1000

bounds = [(-1, 1.5), (0, 2.5), (0, 10), (-10, 10), (-4, 4)]

for x in np.arange(0.50, 0.55, 0.05):
    initial_guess = [0, 0, 0, 0, 0]
    # result = differential_evolution(cost_function, bounds, args=(x,))
    result = minimize(cost_function, initial_guess, x, bounds=bounds)
    best_params = result.x
    best_cost = result.fun
    print(f"{x:.6f} {best_cost:.6f} {best_params}")
    best_results.append([x, best_cost, *best_params])
